# 1 Modeling Volatiliy

Find data for historical prices for any publicly traded equity. To ensure the length of data, use daily data for at least 3 years. Fit the historical data to

1. Geometric Brownian motion
2. Any non-constant volatility model

Find market data for option prices for this stock. Use option prices to

3. Show volatility smile
4. Construct term structure of volatility
5. Plot the volatility surface, as a function of time to maturity and moneyness 

In [363]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pybroker
import yfinance as yf
from pybroker.data import DataSource
from pybroker.ext.data import AKShare
import warnings
from pylab import mpl
from datetime import datetime
from scipy.optimize import minimize
from scipy.stats import norm
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-white')
mpl.rcParams['savefig.dpi'] = 300 
mpl.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False


In [364]:
STRAT_DATE = '2021-01-01'
END_DATE = '2024-10-17'
CURRENT_DATE = END_DATE
r_f = 0.03 

## 1.1 Constant Volatility
If the stock price follows **Geometric Brownian Motion**, then 
$$
\begin{aligned}
d\ln(S)=\left( \mu-\frac{1}{2}\sigma^{2} \right)dt+\sigma dB
\end{aligned}
$$
hence 
$$
\begin{gathered}
\Delta \ln S =\quad\left(\mu-\frac12\sigma^2\right)\Delta t+\sigma\Delta B \\
\ln S(t_i)-\ln S(t_{i-1}) =\quad\left(\mu-\frac12\sigma^2\right)\Delta t+\sigma(B(t_i)-B(t_{i-1})) \\
\frac{\ln S(t_i)-\ln S(t_{i-1})}{\sqrt{\Delta t}} =\quad\left(\mu-\frac12\sigma^2\right)\sqrt{\Delta t}+\sigma\frac{B(t_i)-B(t_{i-1})}{\sqrt{\Delta t}} 
\end{gathered}
$$
here we let $\Delta t=1\text{ day }= 1/252 \text{ year }$ , let $y_{i}=\frac{\ln S(t_{i})-\ln S(t_{i-1})}{\sqrt{ \Delta t }}$, then 
$$
\begin{aligned}
\hat{\sigma}^{2}& =\frac{1}{N-1}\left(\sum_{i=1}^{N}y_{i}^{2}-N\bar{y}^{2}\right) \\
\end{aligned}
$$
and 
$$
\hat{\mu}=\bar{y}+\frac{1}{2}\hat{\sigma}^{2}
$$

In [365]:
# read local data
df = pd.read_csv('MSFT_stock_data.csv', index_col=0)
df.set_index('date', inplace=True)

# calculate log returns
delta_t = 1 / 252
df['y'] = np.log(df['close'] / df['close'].shift(1)) / np.sqrt(delta_t)
df.dropna(inplace=True)


### Statistic Estimations

In [ ]:

# estimate parameters
y_bar = df['y'].mean()
N = len(df)
sigma_stat = np.std(df['y'])
mu_stat = y_bar/np.sqrt(delta_t) + 1 / 2 * sigma_stat ** 2

print(f"sigma_stat = {sigma_stat:.4f}")
print(f"mu_stat = {mu_stat:.4f}")

### Maximum Likelihood Estimation

In [ ]:
def GBM_log_likelihood(params, log_returns):
    mu_gbm, sigma_gbm = params
    n = len(log_returns)
    mu = (mu_gbm - 1/2 * sigma_gbm**2) * np.sqrt(delta_t)
    sigma = sigma_gbm
    log_likelihood = -n/2 * np.log(2 * np.pi * sigma**2) -\
                     1/(2 * sigma**2) * np.sum((log_returns - mu)**2)
    return -log_likelihood

initial_guess = [0.05, 0.25]
results = minimize(GBM_log_likelihood, initial_guess, args=(df['y'], ))
mu_gbm, sigma_gbm = results.x
print(f"sigma_gbm = {sigma_gbm:.4f}")
print(f"mu_gbm = {mu_gbm:.4f}")

## 1.2 GARCH Model

We use GARCH(1, 1) model to fit $y_{i}$, to get our conditional variance

In [ ]:
import arch
garch_model = arch.arch_model(df['y'], vol='garch', p=1, q=1)
garch_fit = garch_model.fit(disp='off')
df['GARCH_volatility'] =(garch_fit.conditional_volatility)
garch_fit.summary()

### Plot Results

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['y'], label='Log Returns')
plt.title('MSFT Log Returns')
plt.xlabel('Date')
plt.ylabel('Log Returns')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df.index, df[['GARCH_volatility']], linestyle='-', color='b')
plt.axhline(y=sigma_stat, color='r', linestyle='--', label='Constant Volatility')
plt.ylim(0)
plt.xlabel('Date')
plt.ylabel('GARCH_volatility')
plt.title('GARCH Volatility Over Time')
plt.xticks(rotation=45, ha='right')
plt.gca().xaxis.set_major_locator(plt.MaxNLocator(10))  # Reduce the number of x-axis labels
plt.grid()
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))

ax1 = plt.gca()
ax1.plot(df.index, df['GARCH_volatility'], linestyle='-', color='b', label='GARCH Volatility')
ax1.axhline(y=sigma_stat, color='r', linestyle='--', label='Constant Volatility')
ax1.set_xlabel('Date')
ax1.set_ylabel('GARCH Volatility')
ax1.tick_params(axis='y', labelcolor='b')  
ax1.grid()
ax1.legend(loc='upper left')    


ax2 = ax1.twinx()
ax2.plot(df.index, df['close'], linestyle='-', color='g', label='Close Price')
ax2.set_ylabel('Close Price')
ax2.tick_params(axis='y', labelcolor='g')  
ax2.legend(loc='upper right')


plt.xticks(rotation=45, ha='right')
ax1.xaxis.set_major_locator(plt.MaxNLocator(10))  


plt.title('GARCH Volatility and Close Price Over Time')
plt.tight_layout()

plt.show()

## 1.3 Volatility Smile

Find the today's call option data of MSFT, there are 15 different options. The expiration ranges from 2024 - 10 - 18 to 2026-12-18

The *TTM* ( time to maturity) needs to be converted to years, so each is divided by 365 

Since there are outliers in *impliedVolatility* 

In [372]:
df_option = pd.read_csv('call_option.csv') #Data was collected on 2024-10-16
current_price = 416.72

# choose the option date
option_columns = ['expiration', 'strike', 'last', 'impliedVolatility']
df_option = df_option[option_columns]
df_option['expiration'] = pd.to_datetime(df_option['expiration'])
df_option['impliedVolatility'] = df_option['impliedVolatility'] / 100

# calculate time to maturity (in years)
df_option['TTM'] = (df_option['expiration'] - pd.Timestamp('2024-10-16')).dt.days 
df_option['TTM'] = df_option['TTM'] / 365 # convert days to years
df_option['moneyness'] = current_price / df_option['strike']

# exclude options with implied volatility changing too dramatically
df_option = df_option[(df_option['impliedVolatility'] > 0.01) & (df_option['impliedVolatility'] < 0.6)]
df_option = df_option[df_option['impliedVolatility'].pct_change() < 0.3]

# exclude options with extreme strike price 
df_option = df_option[(df_option['strike'] > current_price * 0.8) & (df_option['strike'] < current_price * 1.5)]

#exclude options whose expiration near today or too far
df_option = df_option[(df_option['expiration'] > pd.to_datetime('2024-10-19')) & (df_option['expiration'] < pd.to_datetime('2025-12-01'))]

#exclude options that were never traded
df_option = df_option[df_option['last'] > 0.01]


In [ ]:
for i, (expiration, group) in enumerate(df_option.groupby('expiration')):
        
    if i % 5 == 0:
        plt.figure(figsize=(10, 6))
    plt.plot(group['strike'], group['impliedVolatility'], marker='', linestyle='-', label=f"Expiration: {expiration.strftime('%Y-%m-%d')}")
    if i % 5 == 4:
        plt.xlabel('Strike Price')
        plt.ylabel('Implied Volatility')
        plt.title('Implied Volatility vs. Strike Price by Expiration')
        plt.legend()
        plt.grid(True)
        plt.show()

## 1.4 Term Structure of Volatility

In [ ]:
top_15_strikes = df_option['strike'].value_counts().head(15).index
filtered_df = df_option[df_option['strike'].isin(top_15_strikes)]

for i, (strike, group) in enumerate(filtered_df.groupby('strike')):
        
    if i % 5 == 0:
        plt.figure(figsize=(10, 6))
    plt.plot(group['TTM'], group['impliedVolatility'], marker='', linestyle='-', label=f"Strike Price: {strike}")
    if i % 5 == 4:
        plt.xlabel('TTM')
        plt.ylabel('Implied Volatility')
        plt.title('Implied Volatility vs. TTM by Strike')
        plt.legend()
        plt.grid(True)
        plt.show()

## 1.5 Volatility Surface

In [ ]:
from scipy.interpolate import griddata

# Create a grid of TTM and moneyness
TTM_range = np.linspace(df_option['TTM'].min(), df_option['TTM'].max(), 50)
moneyness_range = np.linspace(df_option['moneyness'].min(), df_option['moneyness'].max(), 50)
TTM_grid, moneyness_grid = np.meshgrid(TTM_range, moneyness_range)

# Interpolate implied volatility onto the grid
impliedVolatility_grid = griddata(
    (df_option['TTM'], df_option['moneyness']), 
    df_option['impliedVolatility'], 
    (TTM_grid, moneyness_grid), 
    method='linear'
)

# Plotting with plot_surface
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(TTM_grid, moneyness_grid, impliedVolatility_grid, cmap=plt.get_cmap('viridis'), edgecolor='none')

ax.set_ylabel('Moneyness')
ax.set_xlabel('TTM (Time to Maturity)')
ax.set_zlabel('Implied Volatility')

plt.title('Volatility Surface')
plt.show()


# 2 Vanilla European option pricing

In [ ]:
sigma, mu = [sigma_gbm, mu_gbm]
params = garch_fit.params
a, b, c = params[1], params[2], params[3]
print(f"sigma: {sigma:.4f}, mu: {mu:.4f}")
print(f"a: {a:.4f}, b: {b:.4f}, c: {c:.4f}")

## Monte Carlo

In [377]:
# def GARCH_simulate(row, params):
#     a, b, c = params[1], params[2], params[3]
#     GARCH_volatility = np.sqrt(a + b * row['y']**2 \
#                     + c * row['GARCH_volatility']**2)
#     y = np.random.normal(0, GARCH_volatility)
#     return pd.Series([y, GARCH_volatility], index=['y', 'GARCH_volatility'])
    
# T = 1
# expiration = 29
# for i in range(T):
#     df_simu = df[['y', 'GARCH_volatility']].tail(1)
#     df_simu['y'] = df_simu['y'] -\
#         (mu - 1/2 * df_simu['GARCH_volatility'].iloc[-1] ** 2) * np.sqrt(delta_t)
#     for t in range(expiration):
#         new_row = df_simu.iloc[-1].apply(GARCH_simulate, params)
#         df_simu = pd.concat([df_simu, new_row])

## 2.2 Numerical PDE

In [425]:
# paramaters setting
k = r_f * 2 / sigma**2
alpha = -1/2 * (k - 1)
beta = -1/4 * (k + 1)**2

def PDE_Pricing(TTM, strike, S_0):
    tau = TTM * sigma**2 / 2
    df = pd.DataFrame({'x': np.arange(-1, 1, 0.01)})
    df.set_index('x', inplace=True, drop=True)
    df['0'] = np.maximum(np.exp(1/2 * (k+1) * df.index) - np.exp(1/2 * (k-1) * df.index), 0)

    # set mesh size and set boundary condition
    delta = 0.00001
    for i in range(int(tau / delta)):
        df[f'empty_{i+1}'] = None
    df.iloc[0] = df.iloc[0].fillna(method='ffill')
    df.iloc[-1] = df.iloc[-1].fillna(method='ffill')

    # explicit scheme
    for j in range(1, int(tau / delta) + 1):
        for i in range(1, len(df) - 1):
            df.iloc[i, j] = alpha * df.iloc[i - 1, j - 1] \
                            + (1 - 2 * alpha) * df.iloc[i, j - 1] \
                            + alpha * df.iloc[i + 1, j - 1]
    
    # convert back to call option price 
    df['v'] = np.exp(alpha * df.index + beta * tau) * df.iloc[:,-1]
    df['C'] = strike * df['v']
    df['S'] = strike * np.exp(df.index)
    # return df
    return float(df.loc[(df['S'] - S_0).abs().idxmin()][['C']])

In [428]:
TTMS = df_option['TTM'].unique()
df_PDE = df_option[df_option['TTM'] == TTMS[1]]
strikes = df_PDE['strike'].unique()[:30]
# strikes = [420]
TTM = df_PDE['TTM'].iloc[0]
for strike in strikes:
    # tmp = PDE_Pricing(TTM, strike, S_0=current_price)
    df_PDE.loc[df_PDE['strike']==strike, 'PDE_Price'] = PDE_Pricing(TTM, strike, S_0=current_price)

In [429]:
df_PDE

,expiration,strike,last,impliedVolatility,TTM,moneyness,PDE_Price
225,2024-12-20,335.0,85.029999,0.348614,0.178082,1.243940,82.136795
226,2024-12-20,340.0,83.650002,0.334213,0.178082,1.225647,75.015606
227,2024-12-20,345.0,75.750000,0.330477,0.178082,1.207884,71.946947
228,2024-12-20,350.0,73.000000,0.323570,0.178082,1.190629,64.650992
229,2024-12-20,355.0,62.750000,0.313326,0.178082,1.173859,61.408695
230,2024-12-20,360.0,63.130001,0.308279,0.178082,1.157556,58.091085
231,2024-12-20,365.0,57.799999,0.300463,0.178082,1.141699,50.542855
232,2024-12-20,370.0,53.299999,0.294855,0.178082,1.126270,47.063598
233,2024-12-20,375.0,48.480000,0.287746,0.178082,1.111253,43.513761
234,2024-12-20,380.0,45.900002,0.283524,0.178082,1.096632,35.738762


## 2.3 Binomial Model

### 2.3.1 Cox-Ross_Rubinstein

In [351]:
from math import comb

u = np.exp(sigma * np.sqrt(delta_t))
d = 1 / u
p = (np.exp(r_f * delta_t) - d) / (u - d)

def Binomial_Pricing(row, S, u, d, p):
    price = 0
    days = int(row['TTM'] * 252)
    for i in range(days+1):
        prob = comb(days, i) * p**i * (1-p)**(days-i)
        S_T = u**i * d**(days-i) * S
        price += max(prob * (S_T - row['strike']), 0)
    return price 

df_option['Binomial_CRR'] = df_option.apply(Binomial_Pricing, S=current_price, u=u, d=d, p=p, axis=1)


In [ ]:
# Create a 5x2 layout for displaying the plots with larger fonts
fig, axes = plt.subplots(5, 2, figsize=(20, 30))
axes = axes.flatten()

# Set font size for labels, titles, and legends
font_size = 24

for i, (expiration, group) in enumerate(df_option.groupby('expiration')):
    ax = axes[i]
    ax.plot(group['strike'], group['last'], label='Last Price', linestyle='-', color='blue')
    ax.plot(group['strike'], group['Binomial_CRR'], label='Binomial_CRR Price', linestyle='--', color='red')

    ax.set_xlabel('Strike Price', fontsize=font_size)
    ax.set_ylabel('Price', fontsize=font_size)
    ax.set_title(f'Expiration: {expiration}', fontsize=font_size)
    ax.legend(fontsize=font_size)
    ax.grid(True)

    ax.tick_params(axis='both', which='major', labelsize=font_size)

for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()


### 2.3.2 Jarrow_Rudd

In [353]:
p = 1/2
u = np.exp((r_f - 1/2 * sigma**2) * delta_t + sigma * np.sqrt(delta_t))
d = np.exp((r_f - 1/2 * sigma**2) * delta_t - sigma * np.sqrt(delta_t))
def Binomial_Pricing(row, S, u, d, p):
    price = 0
    days = int(row['TTM'] * 252)
    for i in range(days+1):
        prob = comb(days, i) * p**i * (1-p)**(days-i)
        S_T = u**i * d**(days-i) * S
        price += max(prob * (S_T - row['strike']), 0)
    return price 

df_option['Binomial_JR'] = df_option.apply(Binomial_Pricing, S=current_price, u=u, d=d, p=p, axis=1)

In [ ]:
# Create a 5x2 layout for displaying the plots with larger fonts
fig, axes = plt.subplots(5, 2, figsize=(20, 30))
axes = axes.flatten()

# Set font size for labels, titles, and legends
font_size = 24

for i, (expiration, group) in enumerate(df_option.groupby('expiration')):
    ax = axes[i]
    ax.plot(group['strike'], group['last'], label='Last Price', linestyle='-', color='blue')
    ax.plot(group['strike'], group['Binomial_JR'], label='Binomial_JR Price', linestyle='--', color='red')

    ax.set_xlabel('Strike Price', fontsize=font_size)
    ax.set_ylabel('Price', fontsize=font_size)
    ax.set_title(f'Expiration: {expiration}', fontsize=font_size)
    ax.legend(fontsize=font_size)
    ax.grid(True)

    ax.tick_params(axis='both', which='major', labelsize=font_size)

for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()


## 2.4 Black-Scholes Formula

In [355]:

def BS_formula(row, S, r, sigma):
    
    '''
    use BS formula to calculate option price
    S is stock price of MSFT on 17 Oct, 2024
    r is set as 0.03
    '''
    
    d1 = (np.log(S / row['strike'])\
           + (r + 1/2 * sigma**2)*(row['TTM']))\
              / (sigma * np.sqrt(row['TTM']))
    d2 = d1 - sigma * np.sqrt(row['TTM'])
    return S * norm.cdf(d1) \
        - row['strike'] * np.exp(-r * row['TTM']) * norm.cdf(d2)

df_option['BS_price'] = df_option.apply(BS_formula, S=current_price, r = r_f, sigma=sigma, axis=1)

In [ ]:
# Create a 5x2 layout for displaying the plots with larger fonts
fig, axes = plt.subplots(5, 2, figsize=(20, 30))
axes = axes.flatten()

# Set font size for labels, titles, and legends
font_size = 24

for i, (expiration, group) in enumerate(df_option.groupby('expiration')):
    ax = axes[i]
    ax.plot(group['strike'], group['last'], label='Last Price', linestyle='-', color='blue')
    ax.plot(group['strike'], group['BS_price'], label='BS Price', linestyle='--', color='red')

    ax.set_xlabel('Strike Price', fontsize=font_size)
    ax.set_ylabel('Price', fontsize=font_size)
    ax.set_title(f'Expiration: {expiration}', fontsize=font_size)
    ax.legend(fontsize=font_size)
    ax.grid(True)

    ax.tick_params(axis='both', which='major', labelsize=font_size)

for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()
